In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from tensorboardX import SummaryWriter      

import torchvision
import torchvision.transforms as transforms

from models import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9219/10000 (92%)



In [3]:
#### Prune all the QuantConv2D layers' 90% weights with 1) unstructured, and 2) structured manner.

import torch.nn.utils.prune as prune

for i in range(len(model.features)):

    if isinstance(model.features[i], QuantConv2d):
            prune.ln_structured(model.features[i], name = 'weight', amount = 0.9,dim =0, n=1)


In [4]:
print(list(model.features[40].named_parameters())) # check whether there is mask, weight_org, ...
print(model.features[40].weight) # check whether there are many zeros

[('act_alpha', Parameter containing:
tensor(2.5205, device='cuda:0', requires_grad=True)), ('weight_q', Parameter containing:
tensor([[[[-0.8155, -0.8155, -0.4077],
          [-0.8155,  0.4077, -2.0387],
          [-1.2232,  0.4077, -0.8155]],

         [[ 0.4077,  0.0000, -1.2232],
          [-0.0000, -0.4077,  0.4077],
          [ 0.4077, -1.2232, -0.4077]],

         [[-0.4077, -0.8155,  0.4077],
          [-0.8155, -0.4077,  0.4077],
          [-0.4077, -0.8155, -0.0000]],

         ...,

         [[ 0.8155,  0.8155,  0.0000],
          [ 0.4077,  0.4077,  1.2232],
          [-1.2232, -1.6310, -0.8155]],

         [[-0.8155,  0.4077,  0.8155],
          [-1.2232, -2.4465,  0.4077],
          [-1.2232, -1.6310, -0.8155]],

         [[-0.4077, -0.8155, -2.4465],
          [ 0.4077,  0.8155, -0.8155],
          [-1.2232, -1.2232,  0.4077]]],


        [[[-0.0000, -0.8155, -1.2232],
          [-1.6310, -0.8155,  0.4077],
          [-0.4077, -0.4077,  0.4077]],

         [[-0.0000, -1.6

In [5]:
### Check sparsity ###
mask1 = model.features[40].weight_mask
sparsity_mask1 = (mask1 == 0).sum() / mask1.nelement()

print("Sparsity level: ", sparsity_mask1)

Sparsity level:  tensor(0.9004, device='cuda:0')


In [6]:
## check accuracy after pruning
model.cuda()
model.eval()
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 1000/10000 (10%)



In [7]:
## Start finetuning (training here), and see how much you can recover your accuracy ##
## You can change hyper parameters such as epochs or lr ##
lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0
#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9,weight_decay=weight_decay)
#cudnn.benchmark = True
if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)+"_pruning_structured"
if not os.path.exists(fdir):
    os.makedirs(fdir)
    
for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)
    train(trainloader, model, criterion, optimizer, epoch)
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)
    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec': best_prec,
            'optimizer': optimizer.state_dict(),
        }, is_best, fdir)



Epoch: [0][0/391]	Time 0.620 (0.620)	Data 0.306 (0.306)	Loss 3.0780 (3.0780)	Prec 7.812% (7.812%)
Epoch: [0][100/391]	Time 0.049 (0.060)	Data 0.003 (0.008)	Loss 1.7210 (1.9627)	Prec 35.938% (24.899%)
Epoch: [0][200/391]	Time 0.053 (0.057)	Data 0.002 (0.005)	Loss 1.7438 (1.8629)	Prec 31.250% (28.226%)
Epoch: [0][300/391]	Time 0.059 (0.056)	Data 0.002 (0.004)	Loss 1.8330 (1.8130)	Prec 30.469% (30.015%)
Validation starts
Test: [0/79]	Time 0.365 (0.365)	Loss 1.7083 (1.7083)	Prec 37.500% (37.500%)
 * Prec 33.000% 
best acc: 33.000000
Epoch: [1][0/391]	Time 0.449 (0.449)	Data 0.407 (0.407)	Loss 1.7619 (1.7619)	Prec 30.469% (30.469%)
Epoch: [1][100/391]	Time 0.053 (0.058)	Data 0.002 (0.006)	Loss 1.6597 (1.6544)	Prec 34.375% (35.048%)
Epoch: [1][200/391]	Time 0.049 (0.056)	Data 0.002 (0.004)	Loss 1.7291 (1.6535)	Prec 35.156% (36.011%)
Epoch: [1][300/391]	Time 0.044 (0.055)	Data 0.002 (0.004)	Loss 1.5036 (1.6472)	Prec 36.719% (36.651%)
Validation starts
Test: [0/79]	Time 0.493 (0.493)	Loss 1.53

Epoch: [15][200/391]	Time 0.058 (0.057)	Data 0.002 (0.004)	Loss 1.4857 (1.4839)	Prec 47.656% (44.850%)
Epoch: [15][300/391]	Time 0.053 (0.056)	Data 0.002 (0.003)	Loss 1.7088 (1.4864)	Prec 37.500% (44.884%)
Validation starts
Test: [0/79]	Time 0.377 (0.377)	Loss 1.4847 (1.4847)	Prec 48.438% (48.438%)
 * Prec 45.060% 
best acc: 47.210000
Epoch: [16][0/391]	Time 0.382 (0.382)	Data 0.342 (0.342)	Loss 1.4197 (1.4197)	Prec 43.750% (43.750%)
Epoch: [16][100/391]	Time 0.046 (0.057)	Data 0.002 (0.006)	Loss 1.5512 (1.4770)	Prec 41.406% (44.833%)
Epoch: [16][200/391]	Time 0.060 (0.055)	Data 0.012 (0.004)	Loss 1.3584 (1.4842)	Prec 50.000% (44.912%)
Epoch: [16][300/391]	Time 0.049 (0.055)	Data 0.002 (0.003)	Loss 1.5789 (1.4842)	Prec 43.750% (44.866%)
Validation starts
Test: [0/79]	Time 0.276 (0.276)	Loss 1.4709 (1.4709)	Prec 46.875% (46.875%)
 * Prec 45.210% 
best acc: 47.210000
Epoch: [17][0/391]	Time 0.338 (0.338)	Data 0.291 (0.291)	Loss 1.4062 (1.4062)	Prec 43.750% (43.750%)
Epoch: [17][100/391]	

Epoch: [30][300/391]	Time 0.072 (0.059)	Data 0.003 (0.006)	Loss 1.5631 (1.4513)	Prec 40.625% (46.255%)
Validation starts
Test: [0/79]	Time 0.308 (0.308)	Loss 1.4771 (1.4771)	Prec 46.094% (46.094%)
 * Prec 47.910% 
best acc: 48.200000
Epoch: [31][0/391]	Time 0.425 (0.425)	Data 0.382 (0.382)	Loss 1.4872 (1.4872)	Prec 43.750% (43.750%)
Epoch: [31][100/391]	Time 0.046 (0.059)	Data 0.007 (0.007)	Loss 1.3767 (1.4454)	Prec 50.000% (46.535%)
Epoch: [31][200/391]	Time 0.054 (0.057)	Data 0.002 (0.005)	Loss 1.3358 (1.4483)	Prec 52.344% (46.521%)
Epoch: [31][300/391]	Time 0.041 (0.056)	Data 0.004 (0.005)	Loss 1.8140 (1.4638)	Prec 28.125% (46.153%)
Validation starts
Test: [0/79]	Time 0.384 (0.384)	Loss 1.5379 (1.5379)	Prec 39.844% (39.844%)
 * Prec 44.480% 
best acc: 48.200000
Epoch: [32][0/391]	Time 0.400 (0.400)	Data 0.358 (0.358)	Loss 1.3607 (1.3607)	Prec 49.219% (49.219%)
Epoch: [32][100/391]	Time 0.053 (0.058)	Data 0.002 (0.007)	Loss 1.3661 (1.4682)	Prec 53.906% (45.537%)
Epoch: [32][200/391]	

Validation starts
Test: [0/79]	Time 0.327 (0.327)	Loss 1.5138 (1.5138)	Prec 46.094% (46.094%)
 * Prec 48.920% 
best acc: 48.950000
Epoch: [46][0/391]	Time 0.445 (0.445)	Data 0.398 (0.398)	Loss 1.2840 (1.2840)	Prec 50.781% (50.781%)
Epoch: [46][100/391]	Time 0.054 (0.058)	Data 0.002 (0.007)	Loss 1.4434 (1.4231)	Prec 43.750% (47.850%)
Epoch: [46][200/391]	Time 0.054 (0.057)	Data 0.002 (0.006)	Loss 1.3525 (1.4355)	Prec 48.438% (47.190%)
Epoch: [46][300/391]	Time 0.053 (0.056)	Data 0.004 (0.005)	Loss 1.5156 (1.4368)	Prec 42.188% (47.197%)
Validation starts
Test: [0/79]	Time 0.371 (0.371)	Loss 1.4386 (1.4386)	Prec 44.531% (44.531%)
 * Prec 48.630% 
best acc: 48.950000
Epoch: [47][0/391]	Time 0.370 (0.370)	Data 0.328 (0.328)	Loss 1.5556 (1.5556)	Prec 43.750% (43.750%)
Epoch: [47][100/391]	Time 0.053 (0.058)	Data 0.002 (0.006)	Loss 1.3561 (1.4271)	Prec 50.000% (47.308%)
Epoch: [47][200/391]	Time 0.053 (0.056)	Data 0.002 (0.005)	Loss 1.5391 (1.4377)	Prec 40.625% (47.065%)
Epoch: [47][300/391]	

 * Prec 48.920% 
best acc: 49.140000
Epoch: [61][0/391]	Time 0.543 (0.543)	Data 0.497 (0.497)	Loss 1.4450 (1.4450)	Prec 46.875% (46.875%)
Epoch: [61][100/391]	Time 0.054 (0.060)	Data 0.002 (0.008)	Loss 1.3737 (1.4288)	Prec 50.000% (47.300%)
Epoch: [61][200/391]	Time 0.059 (0.058)	Data 0.003 (0.006)	Loss 1.4236 (1.4312)	Prec 45.312% (47.225%)
Epoch: [61][300/391]	Time 0.053 (0.057)	Data 0.002 (0.005)	Loss 1.5050 (1.4285)	Prec 46.094% (47.394%)
Validation starts
Test: [0/79]	Time 0.335 (0.335)	Loss 1.4926 (1.4926)	Prec 44.531% (44.531%)
 * Prec 47.930% 
best acc: 49.140000
Epoch: [62][0/391]	Time 0.378 (0.378)	Data 0.320 (0.320)	Loss 1.5191 (1.5191)	Prec 37.500% (37.500%)
Epoch: [62][100/391]	Time 0.054 (0.059)	Data 0.002 (0.008)	Loss 1.4770 (1.4294)	Prec 50.781% (47.687%)
Epoch: [62][200/391]	Time 0.053 (0.057)	Data 0.002 (0.006)	Loss 1.5180 (1.4522)	Prec 46.875% (46.856%)
Epoch: [62][300/391]	Time 0.056 (0.057)	Data 0.004 (0.005)	Loss 1.6442 (1.4579)	Prec 42.969% (46.442%)
Validation s

Epoch: [76][100/391]	Time 0.057 (0.061)	Data 0.020 (0.011)	Loss 1.4275 (1.4559)	Prec 46.875% (45.854%)
Epoch: [76][200/391]	Time 0.052 (0.058)	Data 0.002 (0.008)	Loss 1.3756 (1.4495)	Prec 50.000% (46.517%)
Epoch: [76][300/391]	Time 0.079 (0.057)	Data 0.006 (0.006)	Loss 1.5258 (1.4480)	Prec 39.062% (46.468%)
Validation starts
Test: [0/79]	Time 0.394 (0.394)	Loss 1.4530 (1.4530)	Prec 46.094% (46.094%)
 * Prec 48.750% 
best acc: 49.460000
Epoch: [77][0/391]	Time 0.499 (0.499)	Data 0.453 (0.453)	Loss 1.4417 (1.4417)	Prec 46.875% (46.875%)
Epoch: [77][100/391]	Time 0.054 (0.059)	Data 0.002 (0.007)	Loss 1.4769 (1.4520)	Prec 42.188% (46.627%)
Epoch: [77][200/391]	Time 0.062 (0.057)	Data 0.001 (0.005)	Loss 1.4466 (1.4475)	Prec 46.875% (47.093%)
Epoch: [77][300/391]	Time 0.056 (0.056)	Data 0.002 (0.004)	Loss 1.4391 (1.4396)	Prec 48.438% (47.192%)
Validation starts
Test: [0/79]	Time 0.422 (0.422)	Loss 1.4649 (1.4649)	Prec 46.094% (46.094%)
 * Prec 48.450% 
best acc: 49.460000
Epoch: [78][0/391]	

Epoch: [91][200/391]	Time 0.047 (0.057)	Data 0.002 (0.004)	Loss 1.3721 (1.4155)	Prec 56.250% (48.002%)
Epoch: [91][300/391]	Time 0.054 (0.056)	Data 0.002 (0.004)	Loss 1.3509 (1.4103)	Prec 47.656% (48.061%)
Validation starts
Test: [0/79]	Time 0.457 (0.457)	Loss 1.3494 (1.3494)	Prec 50.000% (50.000%)
 * Prec 49.530% 
best acc: 51.080000
Epoch: [92][0/391]	Time 0.535 (0.535)	Data 0.478 (0.478)	Loss 1.4185 (1.4185)	Prec 48.438% (48.438%)
Epoch: [92][100/391]	Time 0.053 (0.062)	Data 0.002 (0.009)	Loss 1.5184 (1.4027)	Prec 45.312% (48.468%)
Epoch: [92][200/391]	Time 0.066 (0.059)	Data 0.002 (0.006)	Loss 1.3316 (1.4125)	Prec 50.781% (47.956%)
Epoch: [92][300/391]	Time 0.054 (0.058)	Data 0.002 (0.005)	Loss 1.3324 (1.4134)	Prec 50.000% (47.924%)
Validation starts
Test: [0/79]	Time 0.392 (0.392)	Loss 1.3474 (1.3474)	Prec 51.562% (51.562%)
 * Prec 49.950% 
best acc: 51.080000
Epoch: [93][0/391]	Time 0.405 (0.405)	Data 0.358 (0.358)	Loss 1.4455 (1.4455)	Prec 43.750% (43.750%)
Epoch: [93][100/391]	

In [8]:
## check your accuracy again after finetuning
PATH = "result/VGG16_quant_pruning_structured/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda")

model.cuda()
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 5108/10000 (51%)



In [9]:
## Send an image and use prehook to grab the inputs of all the QuantConv2d layers

class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
42 -th layer prehooked
47 -th layer prehooked
51 -th layer prehooked
55 -th layer prehooked


In [10]:
##### Find "weight_int" for features[3] ####
w_bit = 4
weight_q = model.features[3].weight_q
w_alpha = model.features[3].weight_quant.wgt_alpha
w_delta = w_alpha /(2**(w_bit-1)-1)

weight_int = weight_q / w_delta
print(weight_int)

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         ...,

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         ...,

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

    

In [11]:
#### check your sparsity for weight_int is near 90% #####
#### Your sparsity could be >90% after quantization #####
sparsity_weight_int = (weight_int == 0).sum() / weight_int.nelement()
print("Sparsity level: ", sparsity_weight_int)

Sparsity level:  tensor(0.0024, device='cuda:0')
